In [ ]:


import numpy as np
from scipy.cluster.hierarchy import ward, fcluster
from itertools import combinations
from collections import defaultdict

# Define the dataset
data = np.array([
    [1, 2],
    [2, 3],
    [4, 5],
    [5, 6],
    [7, 8],
    [8, 9]
])

# Define the transaction dataset
transactions = [
    ['A', 'B', 'C'],
    ['A', 'B'],
    ['C', 'D'],
    ['A', 'C'],
    ['B', 'C'],
    ['A', 'B', 'C']
]

# Perform hierarchical clustering
from scipy.cluster.hierarchy import ward
linkage_matrix = ward(data)
clusters = fcluster(linkage_matrix, 2, criterion='maxclust')

# Group transactions by cluster
clustered_transactions = defaultdict(list)
for i, cluster in enumerate(clusters):
    clustered_transactions[cluster].append(transactions[i])

# Apply Apriori to each cluster
def apriori(transactions, min_support):
    items = set()
    for transaction in transactions:
        items.update(transaction)

    support_counts = defaultdict(int)
    frequent_1_itemsets = []
    for item in items:
        support_count = sum(1 for transaction in transactions if item in transaction)
        if support_count >= min_support:
            frequent_1_itemsets.append((item,))
            support_counts[(item,)] = support_count

    frequent_itemsets = frequent_1_itemsets
    k = 2
    while frequent_1_itemsets:
        candidate_itemsets = set()
        for itemset1 in frequent_1_itemsets:
            for itemset2 in frequent_1_itemsets:
                if itemset1 != itemset2:
                    candidate_itemset = tuple(sorted(set(itemset1) | set(itemset2)))
                    if len(candidate_itemset) == k:
                        candidate_itemsets.add(candidate_itemset)

        frequent_k_itemsets = []
        for candidate_itemset in candidate_itemsets:
            support_count = sum(1 for transaction in transactions if set(candidate_itemset).issubset(transaction))
            if support_count >= min_support:
                frequent_k_itemsets.append(candidate_itemset)
                support_counts[candidate_itemset] = support_count

        frequent_itemsets.extend(frequent_k_itemsets)
        frequent_1_itemsets = frequent_k_itemsets
        k += 1

    return [(itemset, support_counts[itemset]) for itemset in frequent_itemsets]

min_support = 2
for cluster, transactions in clustered_transactions.items():
    frequent_itemsets = apriori(transactions, min_support)
    print(f"Cluster {cluster}:")
    for itemset, support in frequent_itemsets:
        print(itemset, support)
    print()

